In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
lipo_chemberta2_valid2 = pd.read_csv('./chemberta2/results/lipo/chemberta2_valid2_lipo_1_predictions.csv')
lipo_molformer_valid2 = pd.read_csv('./molformer/results/lipo/molformer_lipo_valid2_1.csv')
lipo_molbert_valid2 = pd.read_csv('./molbert/results/lipo/molbert_valid2_lipo_1.csv')

# Load the test data for each model
lipo_chemberta2_test = pd.read_csv('./chemberta2/results/lipo/chemberta2_test_lipo_1_predictions.csv')
lipo_molformer_test = pd.read_csv('./molformer/results/lipo/molformer_lipo_test_1.csv')
lipo_molbert_test = pd.read_csv('./molbert/results/lipo/molbert_test_lipo_1.csv')

train_mean = 2.1741626984126983
train_sd = 1.2060271152247222

# features

lipo_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/lipo/chemberta2_valid2_lipo_1_features.csv')
lipo_chemberta2_features_test = pd.read_csv('./chemberta2/features/lipo/chemberta2_test_lipo_1_features.csv')

lipo_molformer_features_valid2 = pd.read_csv('./molformer/features/lipo/molformer_valid2_lipo_1_features.csv')
lipo_molformer_features_test = pd.read_csv('./molformer/features/lipo/molformer_test_lipo_1_features.csv')

lipo_molbert_features_valid2 = pd.read_csv('./molbert/features/lipo/molbert_valid2_lipo_1_features.csv')
lipo_molbert_features_test = pd.read_csv('./molbert/features/lipo/molbert_test_lipo_1_features.csv')

For Lipo (Regression)

In [2]:
# Preparing the actual and predicted values
# Chemberta2
lipo_chemberta_actual = lipo_chemberta2_test['target'] 
lipo_chemberta_pred = lipo_chemberta2_test['pred_raw']

# Molformer
lipo_molformer_actual = lipo_molformer_test['target']
lipo_molformer_pred = lipo_molformer_test['pred_raw']

# molbert
lipo_molbert_actual = lipo_molbert_test['target']
lipo_molbert_pred = lipo_molbert_test['pred_raw']

In [3]:
# Calculating metrics
lipo_metrics_results = {}

for model_name, actual, pred in [("Chemberta2", lipo_chemberta_actual, lipo_chemberta_pred),
                                 ("Molformer", lipo_molformer_actual, lipo_molformer_pred),
                                 ("Molbert", lipo_molbert_actual, lipo_molbert_pred)]:
    lipo_metrics_results[model_name] = {
        "MAE": mean_absolute_error(actual, pred),
        "RMSE": np.sqrt(mean_squared_error(actual, pred)),
        "R2 Score": r2_score(actual, pred),
        "Correlation": pearsonr(actual, pred)[0]  # Only record the correlation coefficient
    }

lipo_metrics_results

{'Chemberta2': {'MAE': 0.4868551307755036,
  'RMSE': 0.6482952518588416,
  'R2 Score': 0.707140489497853,
  'Correlation': 0.8472657204543115},
 'Molformer': {'MAE': 0.49439111716666667,
  'RMSE': 0.6626307207706376,
  'R2 Score': 0.6940455432003805,
  'Correlation': 0.8377379022328875},
 'Molbert': {'MAE': 0.5154242747119047,
  'RMSE': 0.6940250633438312,
  'R2 Score': 0.6643675374778397,
  'Correlation': 0.8171700472906417}}

In [4]:
# Create the features for the ensemble from the prediction probabilities of being in class 1
lipo_X_ensemble_valid2 = pd.concat([
    lipo_molformer_valid2['pred_z'], 
    lipo_molbert_valid2['pred_z'],
    # add features from training set
    lipo_molformer_features_valid2.iloc[:, 2:],
    lipo_molbert_features_valid2.iloc[:, 2:]
], axis=1)

# change feature names of the ensemble so that they are unique
lipo_X_ensemble_valid2.columns = ['molformer', 'molbert'] + list(lipo_molformer_features_valid2.columns[2:]) + list(lipo_molbert_features_valid2.columns[2:])

# Use the actual labels from any of the models (assuming they are all the same across datasets)
lipo_y_ensemble_valid2 = (lipo_chemberta2_valid2['target'] - train_mean)/train_sd

lipo_X_ensemble_test = pd.concat([
    lipo_molformer_test['pred_z'],  
    lipo_molbert_test['pred_z'],
    # add features from test set
    lipo_molformer_features_test.iloc[:, 2:],
    lipo_molbert_features_test.iloc[:, 2:]
], axis=1)

# change feature names of the ensemble so that they are unique
lipo_X_ensemble_test.columns = ['molformer', 'molbert'] + list(lipo_molformer_features_test.columns[2:]) + list(lipo_molbert_features_test.columns[2:])

# Use the actual labels from any of the models (assuming they are all the same across datasets)
lipo_y_ensemble_test = lipo_chemberta2_test['target']

In [5]:
# scale the features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
lipo_X_ensemble_valid2_scaled = scaler.fit_transform(lipo_X_ensemble_valid2)
lipo_X_ensemble_test_scaled = scaler.transform(lipo_X_ensemble_test)

lipo_X_ensemble_valid2_scaled = pd.DataFrame(lipo_X_ensemble_valid2_scaled, columns=lipo_X_ensemble_valid2.columns)
lipo_X_ensemble_test_scaled = pd.DataFrame(lipo_X_ensemble_test_scaled, columns=lipo_X_ensemble_test.columns)

In [6]:
# lasso model
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize the LassoCV model
lasso_cv = LassoCV(cv=5, max_iter = 5000, random_state=0)

# Fit the model
lasso_cv.fit(lipo_X_ensemble_valid2_scaled, lipo_y_ensemble_valid2)

# Predict the test set
lipo_lasso_pred = lasso_cv.predict(lipo_X_ensemble_test_scaled) * train_sd + train_mean

# Calculate the metrics
lipo_lasso_metrics = {
    "MAE": mean_absolute_error(lipo_y_ensemble_test, lipo_lasso_pred),
    "RMSE": np.sqrt(mean_squared_error(lipo_y_ensemble_test, lipo_lasso_pred)),
    "R2 Score": r2_score(lipo_y_ensemble_test, lipo_lasso_pred),
    "Correlation": pearsonr(lipo_y_ensemble_test, lipo_lasso_pred)[0]
}

lipo_lasso_metrics

{'MAE': 0.47354072834311545,
 'RMSE': 0.6277061072499539,
 'R2 Score': 0.7254468955586539,
 'Correlation': 0.8536592250654939}

In [7]:
# use lasso to select features
coefs = pd.Series(lasso_cv.coef_, index=lipo_X_ensemble_valid2.columns)

selected_features = coefs[coefs != 0].index.tolist()
lipo_X_ensemble_valid2_selected = lipo_X_ensemble_valid2_scaled[selected_features]
lipo_X_ensemble_test_selected = lipo_X_ensemble_test_scaled[selected_features]

print("Selected Features:", selected_features)
print("Filtered Dataset Shape:", lipo_X_ensemble_valid2_selected.shape)

# check how many features have Chemberta2, Molformer, Molbert
selected_features = lipo_X_ensemble_valid2_selected.columns
chemberta_count = sum('chemberta' in feature for feature in selected_features)
molformer_count = sum('molformer' in feature for feature in selected_features)
molbert_count = sum('molbert' in feature for feature in selected_features)

print("Chemberta2 count:", chemberta_count)
print("Molformer count:", molformer_count)
print("Molbert count:", molbert_count)

Selected Features: ['molformer', 'molbert', 'molformer_feature_6', 'molformer_feature_68', 'molformer_feature_78', 'molformer_feature_145', 'molformer_feature_183', 'molformer_feature_191', 'molformer_feature_206', 'molformer_feature_225', 'molformer_feature_238', 'molformer_feature_251', 'molformer_feature_270', 'molformer_feature_325', 'molformer_feature_336', 'molformer_feature_365', 'molformer_feature_366', 'molformer_feature_381', 'molformer_feature_450', 'molformer_feature_476', 'molformer_feature_482', 'molformer_feature_492', 'molformer_feature_499', 'molformer_feature_544', 'molformer_feature_553', 'molformer_feature_566', 'molformer_feature_648', 'molformer_feature_652', 'molformer_feature_660', 'molbert_features_12', 'molbert_features_38', 'molbert_features_39', 'molbert_features_81', 'molbert_features_115', 'molbert_features_186', 'molbert_features_188', 'molbert_features_196', 'molbert_features_204', 'molbert_features_248', 'molbert_features_261', 'molbert_features_277', '

In [8]:
lipo_X_ensemble_valid2_selected = lipo_X_ensemble_valid2_scaled[selected_features]
lipo_X_ensemble_test_selected = lipo_X_ensemble_test_scaled[selected_features]

# check shapes
print(lipo_X_ensemble_valid2_selected.shape)
print(lipo_X_ensemble_test_selected.shape)

(840, 70)
(420, 70)


In [9]:
# Initialize and train the SVR model
from sklearn.svm import SVR

lipo_svr_model = SVR()
lipo_svr_model.fit(lipo_X_ensemble_valid2_selected, lipo_y_ensemble_valid2)

# Predict the test set
lipo_svr_pred = lipo_svr_model.predict(lipo_X_ensemble_test_selected) * train_sd + train_mean

# Calculate the metrics
lipo_svr_metrics = {
    "MAE": mean_absolute_error(lipo_y_ensemble_test, lipo_svr_pred ),
    "RMSE": np.sqrt(mean_squared_error(lipo_y_ensemble_test, lipo_svr_pred )),
    "R2 Score": r2_score(lipo_y_ensemble_test, lipo_svr_pred ),
    "Correlation": pearsonr(lipo_y_ensemble_test, lipo_svr_pred )[0]  # Only record the correlation coefficient
}

lipo_svr_metrics

{'MAE': 0.5066787931152293,
 'RMSE': 0.6744083782354658,
 'R2 Score': 0.6830727630606035,
 'Correlation': 0.8285411981303866}

In [10]:
# initailize and use a 5-fold cross-validation to tune the hyperparameters of a random forest model for regression
from sklearn.ensemble import RandomForestRegressor

lipo_rf_model = RandomForestRegressor(random_state=0)

lipo_rf_model.fit(lipo_X_ensemble_valid2_selected, lipo_y_ensemble_valid2)

# Predict the test set
lipo_rf_best_pred = lipo_rf_model.predict(lipo_X_ensemble_test_selected) * train_sd + train_mean

# Calculate the metrics
lipo_rf_best_metrics = {
    "MAE": mean_absolute_error(lipo_y_ensemble_test, lipo_rf_best_pred),
    "RMSE": np.sqrt(mean_squared_error(lipo_y_ensemble_test, lipo_rf_best_pred)),
    "R2 Score": r2_score(lipo_y_ensemble_test, lipo_rf_best_pred),
    "Correlation": pearsonr(lipo_y_ensemble_test, lipo_rf_best_pred)[0]  # Only record the correlation coefficient
}

lipo_rf_best_metrics

{'MAE': 0.4865154761904762,
 'RMSE': 0.6467595128057738,
 'R2 Score': 0.7085263490284777,
 'Correlation': 0.8425412824428796}

In [11]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Define the hyperparameter space using continuous distributions
lipo_xgb_hyperopt_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 50),
    'max_depth': hp.quniform('max_depth', 3, 7, 2),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# Correctly define the RMSE scorer function
def rmse_scorer(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Convert float outputs of hp.quniform to int for certain parameters
def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    model = xgb.XGBRegressor(**params, random_state=0)
    
    # Cross-validated RMSE as the objective
    score = cross_val_score(model, lipo_X_ensemble_valid2_selected, lipo_y_ensemble_valid2, 
                            scoring=make_scorer(rmse_scorer, greater_is_better=False), cv=5)
    
    # Minimize the positive RMSE (already negative from scoring)
    return {'loss': -score.mean(), 'status': STATUS_OK}

# Run the Bayesian optimization
trials = Trials()
lipo_xgb_best_params = fmin(fn=objective, 
                            space=lipo_xgb_hyperopt_space, 
                            algo=tpe.suggest, 
                            max_evals=100, 
                            trials=trials,
                            early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", lipo_xgb_best_params)


 17%|█▋        | 17/100 [00:19<01:35,  1.15s/trial, best loss: 0.5054138638581215]
Best hyperparameters: {'colsample_bytree': 0.9784591160877237, 'learning_rate': 0.11772376982873545, 'max_depth': 4.0, 'n_estimators': 200.0, 'subsample': 0.904384496926927}


In [12]:
# fit the model with the best hyperparameters
# Convert parameters obtained from Hyperopt to the correct data type
lipo_xgb_best_params['n_estimators'] = int(lipo_xgb_best_params['n_estimators'])
lipo_xgb_best_params['max_depth'] = int(lipo_xgb_best_params['max_depth'])

# Initialize and train the XGBoost model with the best parameters
lipo_xgb_model = xgb.XGBRegressor(**lipo_xgb_best_params, random_state=0)
lipo_xgb_model.fit(lipo_X_ensemble_valid2_selected, lipo_y_ensemble_valid2)

# Predict the test set
lipo_xgb_best_pred = lipo_xgb_model.predict(lipo_X_ensemble_test_selected) * train_sd + train_mean

# Calculate the metrics
lipo_xgb_best_metrics = {
    "MAE": mean_absolute_error(lipo_y_ensemble_test, lipo_xgb_best_pred),
    "RMSE": np.sqrt(mean_squared_error(lipo_y_ensemble_test, lipo_xgb_best_pred)),
    "R2 Score": r2_score(lipo_y_ensemble_test, lipo_xgb_best_pred),
    "Correlation": pearsonr(lipo_y_ensemble_test, lipo_xgb_best_pred)[0]  # Only record the correlation coefficient
}

lipo_xgb_best_metrics

{'MAE': 0.4888723145439511,
 'RMSE': 0.6406112693339926,
 'R2 Score': 0.7140416390668676,
 'Correlation': 0.8456534484965471}

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

torch.manual_seed(0)

# Define RMSE loss
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Define the neural network model for regression
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(1, num_layers):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Hyperparameter space with hp.quniform for integer distribution
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

# Global dataset variables assumed to be defined externally
X = lipo_X_ensemble_valid2_selected
y = lipo_y_ensemble_valid2

# Objective function for Bayesian optimization
def objective(params):
    params['num_layers'] = int(params['num_layers'])  # Ensure num_layers is an integer
    params['num_neurons'] = int(params['num_neurons'])  # Ensure num_neurons is an integer
    kf = KFold(n_splits=5)
    rmse_scores = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Convert DataFrame to numpy arrays before making them PyTorch tensors
        train_dataset = TensorDataset(torch.tensor(X_train.values.astype(np.float32)), 
                                      torch.tensor(y_train.values.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=params['num_layers'],
                         num_neurons=params['num_neurons'], dropout_rate=params['dropout_rate'])
        criterion = RMSELoss()
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            val_preds = model(torch.tensor(X_val.values.astype(np.float32))).squeeze(1)
            val_targets = torch.tensor(y_val.values.astype(np.float32))
            rmse = np.sqrt(mean_squared_error(val_targets.numpy(), val_preds.numpy()))
            rmse_scores.append(rmse)

    avg_rmse = np.mean(rmse_scores)
    return {'loss': avg_rmse, 'status': STATUS_OK} # Minimize RMSE

# Run Bayesian optimization
trials = Trials()
lipo_nn_best_params = fmin(fn=objective,
                           space=space,
                           algo=tpe.suggest,
                           max_evals=50,
                           trials=trials,
                           early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", lipo_nn_best_params)


 26%|██▌       | 13/50 [09:53<28:10, 45.68s/trial, best loss: 0.4900858402252197]
Best hyperparameters: {'dropout_rate': 0.15759648071074484, 'learning_rate': 0.0006142127810276309, 'num_layers': 3.0, 'num_neurons': 236.0}


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

torch.manual_seed(0)

# Define the neural network model again
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(1, num_layers):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Define a function to compute RMSE
def compute_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Convert parameters to the correct format if necessary
lipo_nn_best_params = {
    'num_layers':  int(lipo_nn_best_params['num_layers']),  # Extracted from Bayesian optimization results
    'num_neurons':  int(lipo_nn_best_params['num_neurons']),  # Extracted from Bayesian optimization results
    'dropout_rate': lipo_nn_best_params['dropout_rate'],  # Extracted from Bayesian optimization results
    'learning_rate': lipo_nn_best_params['learning_rate']  # Extracted from Bayesian optimization results
}

# Prepare datasets
X_train_tensor = torch.tensor(lipo_X_ensemble_valid2_selected.values.astype(np.float32))
y_train_tensor = torch.tensor(lipo_y_ensemble_valid2.values.astype(np.float32)).unsqueeze(1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_test_tensor = torch.tensor(lipo_X_ensemble_test_selected.values.astype(np.float32))
y_test_tensor = torch.tensor(lipo_y_ensemble_test.values.astype(np.float32)).unsqueeze(1)

# Initialize the model
model = SimpleNN(input_size=lipo_X_ensemble_valid2_selected.shape[1], num_layers=lipo_nn_best_params['num_layers'],
                         num_neurons=lipo_nn_best_params['num_neurons'], dropout_rate=lipo_nn_best_params['dropout_rate'])
criterion = RMSELoss()
optimizer = optim.Adam(model.parameters(), lr=lipo_nn_best_params['learning_rate'])

# Training loop
model.train()
for epoch in range(100):  # Number of epochs can be adjusted
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluation on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = outputs.squeeze(1).numpy() * train_sd + train_mean

    # Calculate metrics
    mae = mean_absolute_error(y_test_tensor.numpy(), predictions)
    rmse = compute_rmse(y_test_tensor.numpy(), predictions)
    r2 = r2_score(y_test_tensor.numpy(), predictions)
    correlation, _ = pearsonr(y_test_tensor.numpy().squeeze(1), predictions)

    lipo_nn_metrics = {
        'MAE': mae,
        'RMSE': rmse,
        'R2 Score': r2,
        'Correlation': correlation
    }

lipo_nn_metrics

{'MAE': 0.49334157,
 'RMSE': 0.6508812,
 'R2 Score': 0.7047994911061174,
 'Correlation': 0.8397648504764577}

In [15]:
# create a table to record all metrics for lipo
lipo_metrics_results["LASSO"] = lipo_lasso_metrics
lipo_metrics_results["SVR"] = lipo_svr_metrics
lipo_metrics_results["Random Forest"] = lipo_rf_best_metrics
lipo_metrics_results["XGBoost"] = lipo_xgb_best_metrics
lipo_metrics_results["Neural Network"] = lipo_nn_metrics

lipo_metrics_df = pd.DataFrame(lipo_metrics_results).T
# keep 3 digits after the decimal point
lipo_metrics_df = lipo_metrics_df.round(3)

# export table to csv
lipo_metrics_df.to_csv('./split1_lipo_metrics_molformer+molbert.csv')